In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


In [ ]:
# Set file path
file_path = '/content/drive/MyDrive/CIS 550/Project/police_killings_MPV.csv'

In [ ]:
# Load data into data
police_killings_MPV = pd.read_csv(file_path)

In [ ]:
# Create a year column which will be derived from the date entry
police_killings_MPV.insert(1, "Year", police_killings_MPV["Date of Incident (month/day/year)"])
police_killings_MPV.head()

ValueError: ignored

In [ ]:
# Update the new year column to just contain the 4-digit year of the date
police_killings_MPV = pd.read_csv(file_path)
if not ("Year" in police_killings_MPV):
  police_killings_MPV.insert(0, "Year", police_killings_MPV["Date of Incident (month/day/year)"])
  police_killings_MPV.head()
  for i in range(police_killings_MPV["Year"].size):
    date = police_killings_MPV["Year"][i]
    police_killings_MPV["Year"][i] = int(date[date.index("/", -5)+1:])

police_killings_MPV.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,Agency responsible for death,ORI Agency Identifier (if available),Cause of death,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Criminal Charges?,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed/Did Not Have an Actual Weapon,Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),MPV ID,Fatal Encounters ID
0,2020,Name withheld by police,Unknown,Male,Unknown race,NaN,6/9/2020,337 Garrison Place,Virginia Beach,VA,23452.0,Virginia Beach City,Virginia Beach Police Department,VA1280000,Gunshot,Police were called at 4:26 a.m. for a domestic...,Pending investigation,No known charges,https://www.wtkr.com/news/domestic-incident-le...,No,Allegedly Armed,knife,attack,Not fleeing,NaN,NaN,NaN,NaN,8448,28737.0
1,2020,Name withheld by police,Unknown,Male,Unknown race,NaN,5/9/2020,4900 South Lavergne Avenue,Chicago,IL,60638.0,Cook,Chicago Police Department,ILCPD0000,Gunshot,A man allegedly stabbed someone and then alleg...,Pending investigation,No known charges,https://www.chicagotribune.com/news/breaking/c...,No,Allegedly Armed,knife,attack,Not fleeing,NaN,NaN,NaN,NaN,8446,28734.0
2,2020,Name withheld by police,61,Male,Unknown race,NaN,5/9/2020,2118 Fredericksburg Rd,San Antonio,TX,78201.0,Bexar,San Antonio Police Department,TXSPD0000,Gunshot,A man was detained by an off-duty Floresville ...,Pending investigation,No known charges,https://news4sanantonio.com/news/local/police-...,No,Allegedly Armed,knife,attack,Not fleeing,NaN,NaN,off-duty,NaN,8447,28735.0
3,2020,Name withheld by police,22,Male,Unknown race,NaN,5/9/2020,2500 Logistics Drive,Battle Creek,MI,49037.0,Calhoun,"Battle Creek Police Department, Calhoun County...",MI1323700;MI1311300,Gunshot,Following a 15-minute pursuit after two Battle...,Pending investigation,No known charges,https://www.battlecreekenquirer.com/story/news...,Drug or alcohol use,Vehicle,vehicle,attack,Car,NaN,NaN,NaN,NaN,8449,28736.0
4,2020,Steven D. Smith,33,Male,Unknown race,NaN,4/9/2020,200 N. State Street,Syracuse,NY,13203.0,Onondaga,Syracuse Police Department,NY0330100,Gunshot,Officers responded to the Sunoco Gas Station f...,Pending investigation,No known charges,https://cnycentral.com/news/local/large-police...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,NaN,8444,28726.0


In [ ]:
# Drop rows where City is null, since City is part of primary key
police_killings_MPV = police_killings_MPV[police_killings_MPV["City"].notnull()]
print("Number of null cities: ", police_killings_MPV["City"].isnull().sum())

# Drop rows where County is null, since County will be used to join with other datasets
police_killings_MPV = police_killings_MPV[police_killings_MPV["County"].notnull()]
print("Number of null counties: ", police_killings_MPV["County"].isnull().sum())

# Drop rows where Victim name is "Name withhel dby police", since this is being used in primary key
police_killings_MPV = police_killings_MPV[police_killings_MPV["Victim's name"] != "Name withheld by police"]

# NOTE: From EDA, this is only dropping 20 rows, so very minimal data loss


Number of null cities:  0
Number of null counties:  0


In [ ]:
# Add a "Killed by gun wound column"

if not ("is_gun_death" in police_killings_MPV):
  police_killings_MPV.insert(1, "is_gun_death", False)
police_killings_MPV["is_gun_death"] = police_killings_MPV["Cause of death"].str.contains("Gunshot")

# police_killings_MPV["is_gun_death"] = police_killings_MPV["is_gun_death"].astype(int)
police_killings_MPV = police_killings_MPV.replace({'is_gun_death': {True: 'y', False: 'n'}})

# print("Number of gunshot deaths: ", police_killings_MPV["is_gun_death"].sum() )


In [ ]:
# Lits races
r = police_killings_MPV["Victim's race"].unique()
r

array(['Unknown race', 'White', 'Black', 'Hispanic', 'Native American',
       'Asian', 'Pacific Islander'], dtype=object)

First, we create a `Series` that maps each country name to an integer. 


In [ ]:
# TODO: Do we want to replace city/county names with an integer index to improve performance?

all_cities = police_killings_MPV["City"].unique()
city_codes = pd.Series(index=all_cities, data=np.arange(len(all_cities)))
city_codes

# police_killings_MPV["City"] = country_codes.loc[police_killings_MPV["City"]].values
# police_killings_MPV["City"] = police_killings_MPV["City"].astype(np.int64)

Syracuse             0
Lucerne Valley       1
Lacey                2
Bettendorf           3
Washington           4
                  ... 
Belding           3018
Wenonah           3019
Edwardsville      3020
Westwood Lakes    3021
Gordon            3022
Length: 3023, dtype: int64

In [ ]:
# Check whether the columns "Year", "City", "Victim's name" can be used as Primary Keys

# Step 1: Group dataframe by candidate columns 
grouped = police_killings_MPV.groupby(["Year", "City", "Victim's name"]) 
# Step 2: Count rows in each group
counts = grouped.size()

# Note, since we are using Victim's name in the primary key, we should remove all cases where name is "Name withheld by police"
print(counts[counts > 1])

# Note that since no counts are greater than 1, we are able to use these columns as the primary key

Series([], dtype: int64)


In [ ]:
police_killings_MPV.head()

# Drop unwanted columns
police_killings_MPV_reduced_columns = police_killings_MPV.drop(["URL of image of victim", "Street Address of Incident", "Zipcode", "Agency responsible for death", "ORI Agency Identifier (if available)", "A brief description of the circumstances surrounding the death", "Official disposition of death (justified or other)", "Criminal Charges?", "Symptoms of mental illness?", "Unarmed/Did Not Have an Actual Weapon", "Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)", "Alleged Threat Level (Source: WaPo)", "Fleeing (Source: WaPo)", "Body Camera (Source: WaPo)", "WaPo ID (If included in WaPo database)", "Off-Duty Killing?", "Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )", "MPV ID", "Fatal Encounters ID"], axis=1)

# Rename columns for consistency
police_killings_MPV_reduced_columns_renamed = police_killings_MPV_reduced_columns.rename(columns={"Year": "year", "Victim's age": "age", "Victim's gender": "sex", "Victim's name": "name", "Victim's race": "race", "Date of Incident (month/day/year)": "date", "City": "city", "State": "state", "County": "county", "Cause of death": "cause_of_death", "Link to news article or photo of official document": "article"})

police_killings_MPV_reduced_columns_renamed.head()

,year,is_gun_death,name,age,sex,race,date,city,state,county,cause_of_death,article
4,2020,y,Steven D. Smith,33,Male,Unknown race,4/9/2020,Syracuse,NY,Onondaga,Gunshot,https://cnycentral.com/news/local/large-police...
5,2020,y,Joshua Beedie,38,Male,White,4/9/2020,Lucerne Valley,CA,San Bernardino,Gunshot,https://www.vvdailypress.com/story/news/2020/0...
6,2020,y,Michael Forest Reinoehl,48,Male,White,3/9/2020,Lacey,WA,Thurston,Gunshot,https://komonews.com/news/local/authorities-su...
7,2020,y,Timothy Clevenger,53,Male,White,3/9/2020,Bettendorf,IA,Scott,Gunshot,https://www.kwqc.com/2020/09/03/investigation-...
9,2020,y,Deon Kay,18,Male,Black,2/9/2020,Washington,DC,District of Columbia,Gunshot,https://www.wusa9.com/article/news/crime/offic...


In [ ]:
# Set empty blocks to NULL for importing using MySQL Workbench
police_killings_MPV_reduced_columns_renamed.isnull().any()

police_killings_MPV_reduced_columns_renamed["article"].map(lambda x: len(x)).max()

# police_killings_MPV_reduced_columns.loc[police_killings_MPV_reduced_columns['Link to news article or photo of official document'].str.contains(";"), 'Link to news article or photo of official document'] = "NULL"

312

In [ ]:
grouped = police_killings_MPV_reduced_columns_renamed.groupby(["year", "city", "name"]) 
# Step 2: Count rows in each group
counts = grouped.size()

# Note, since we are using Victim's name in the primary key, we should remove all cases where name is "Name withheld by police"
print(counts[counts > 1])

Series([], dtype: int64)


In [ ]:
police_killings_MPV_reduced_columns_renamed = police_killings_MPV_reduced_columns_renamed[['year', 'city', 'name', 'age', 'sex', 'race', 'cause_of_death', 'date', 'county', 'state', 'is_gun_death', 'article']]
police_killings_MPV_reduced_columns_renamed.head()

# Change Male and Female to M and F
sex_map = {
    'Male': 'M',
    'Female': 'F'
}

police_killings_MPV_reduced_columns_renamed_1 = police_killings_MPV_reduced_columns_renamed.replace({"sex": sex_map})
police_killings_MPV_reduced_columns_renamed_1.head()

race_map = {
    'Unknown race': 'Unknown'
}

police_killings_MPV_reduced_columns_renamed_2 = police_killings_MPV_reduced_columns_renamed_1.replace({"race": race_map})
police_killings_MPV_reduced_columns_renamed_2.head()


,year,city,name,age,sex,race,cause_of_death,date,county,state,is_gun_death,article
4,2020,Syracuse,Steven D. Smith,33,M,Unknown,Gunshot,4/9/2020,Onondaga,NY,y,https://cnycentral.com/news/local/large-police...
5,2020,Lucerne Valley,Joshua Beedie,38,M,White,Gunshot,4/9/2020,San Bernardino,CA,y,https://www.vvdailypress.com/story/news/2020/0...
6,2020,Lacey,Michael Forest Reinoehl,48,M,White,Gunshot,3/9/2020,Thurston,WA,y,https://komonews.com/news/local/authorities-su...
7,2020,Bettendorf,Timothy Clevenger,53,M,White,Gunshot,3/9/2020,Scott,IA,y,https://www.kwqc.com/2020/09/03/investigation-...
9,2020,Washington,Deon Kay,18,M,Black,Gunshot,2/9/2020,District of Columbia,DC,y,https://www.wusa9.com/article/news/crime/offic...


In [ ]:
# police_killings_MPV_reduced_columns_renamed.to_csv("police_killings_MPV_processed.csv", index=False, sep=";")
police_killings_MPV_reduced_columns_renamed_2.to_csv("mpv.csv", index=False, sep=";")

In [ ]:
from google.colab import files
files.download('mpv.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DEMOGRAPHIC PROCESSING

In [ ]:
# Set file path
file_path = '/content/drive/MyDrive/CIS 550/Project/demographics.csv'
data = pd.read_csv(file_path)


In [ ]:
data_1 = data.rename(columns={"ORIGIN": "origin", "NAME": "state", "SEX": "sex", "RACE": "race", "AGE": "age", "POPESTIMATE2010": "2010", "POPESTIMATE2011": "2011", "POPESTIMATE2012": "2012", "POPESTIMATE2013": "2013", "POPESTIMATE2014": "2014", "POPESTIMATE2015": "2015", "POPESTIMATE2016": "2016", "POPESTIMATE2017": "2017", "POPESTIMATE2018": "2018", "POPESTIMATE2019": "2019"})
data_1.head()

,SUMLEV,REGION,DIVISION,STATE,state,sex,origin,race,age,CENSUS2010POP,ESTIMATESBASE2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,40,3,6,1,Alabama,0,0,1,0,37991,37991,37818,38678,37824,36812,37575,37698,37629,37526,36370,35594
1,40,3,6,1,Alabama,0,0,1,1,38150,38150,38036,37719,38597,37854,36968,37882,37732,37904,37641,36679
2,40,3,6,1,Alabama,0,0,1,2,39738,39738,39570,38202,37653,38528,37769,37123,38048,37975,38172,37904
3,40,3,6,1,Alabama,0,0,1,3,39827,39828,39805,39690,38113,37716,38605,37896,37266,38170,38197,38510
4,40,3,6,1,Alabama,0,0,1,4,39353,39353,39493,39872,39628,38122,37751,38625,37954,37322,38312,38476


In [ ]:
state_map = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

data_2 = data_1.replace({"state": state_map})
data_2.head()

sex_map = {
    0: 'Total',
    1: 'M',
    2: 'F'
}

data_3 = data_2.replace({"sex": sex_map})
data_3.head()


race_map = {
    1: 'White',
    2: 'Black',
    3: 'Native American',
    4: 'Asian',
    5: 'Pacific Islander',
    6: 'Two or more'
}

data_4 = data_3.replace({"race": race_map})
data_4.head()

data_5 = data_4.drop(["SUMLEV", "REGION", "DIVISION", "STATE", "CENSUS2010POP", "ESTIMATESBASE2010"], axis=1)
data_5.head()

origin_map = {
    0: 'Total',
    1: 'non_hispanic',
    2: 'hispanic'
}

data_6 = data_5.replace({"origin": origin_map})
data_6.head()

,state,sex,origin,race,age,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AL,Total,Total,White,0,37818,38678,37824,36812,37575,37698,37629,37526,36370,35594
1,AL,Total,Total,White,1,38036,37719,38597,37854,36968,37882,37732,37904,37641,36679
2,AL,Total,Total,White,2,39570,38202,37653,38528,37769,37123,38048,37975,38172,37904
3,AL,Total,Total,White,3,39805,39690,38113,37716,38605,37896,37266,38170,38197,38510
4,AL,Total,Total,White,4,39493,39872,39628,38122,37751,38625,37954,37322,38312,38476


In [ ]:
grouped = data_6.groupby(["origin", "state", "sex", "race", "age"]) 
# Step 2: Count rows in each group
counts = grouped.size()

# Note, since we are using Victim's name in the primary key, we should remove all cases where name is "Name withheld by police"
print(counts[counts > 1])

Series([], dtype: int64)


In [ ]:
data_6.to_csv("dem.csv", index=False, sep=";")
from google.colab import files
files.download('dem.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>